# 2.1 Aggregation

Aggregate functions are used to calculate expressions over a set of values.

RDFox offers several, including common functions like SUM and COUNT, and more specialized functions like COUNT_MAX and MAX_ARGMIN.

See the [full list shown here](https://oxfordsemtech.github.io/DocumentationDevBuild/querying.html#aggregate-functions) (excluding non-deterministic functions).

## Using aggregates

To use aggregation in a rule, you must declare it within the body with `AGGREGATE()`, that contains the relevant atoms.

You must also indicate which variable(s) the you're aggregating on with `ON` (known as group variables), and bind the result to a new variable with `BIND`.

## Example

Below is an example of COUNT_MAX used to calculate the number of products with 5-star reviews in a catalogue, assuming 5-stars is the highest rating and that at least one has a 5-star rating.

In [9]:
agg_data = """
@prefix : <https://rdfox.com/example#> .

:catalogueXYZ :hasProduct :productA ,
                    :productB ,
                    :productC .

:productA a :Product ;
    :hasReview :reviewA1 ;
    :hasReview :reviewA2 .

:productB a :Product ;
    :hasReview :reviewB1 .

:productC a :Product ;
    :hasReview :reviewC1 .

:reviewA1 a :Review ;
    :hasStars 5 .

:reviewA2 a :Review ;
    :hasStars 4 .

:reviewB1 a :Review ;
    :hasStars 5 .

:reviewC2 a :Review ;
    :hasStars 3 .
    
"""

In [10]:
agg_rules = """

[?catalogue, :hasNumberOfFiveStars, ?countOfFiveStarProducts] :-
    AGGREGATE (
        [?catalogue, :hasProduct, ?product],
        [?product, :hasReview, ?review],
        [?review, :hasStars, ?stars]
        ON ?catalogue
        BIND COUNT_MAX(?stars) AS ?countOfFiveStarProducts 
    ).
    
"""

In [11]:
import requests

# Set up the SPARQL endpoint
rdfox_server = "http://localhost:12110"

# Helper function to raise exception if the REST endpoint returns an unexpected status code
def assert_response_ok(response, message):
    if not response.ok:
        raise Exception(
            message + "\nStatus received={}\n{}".format(response.status_code, response.text))

# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Add data
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=agg_data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get rules
rules_response = requests.post(rdfox_server + "/datastores/default/content", data=agg_rules)
assert_response_ok(rules_response, "Failed to add rule.")

# Get and issue select query
with open("../queries/2_1-AggregationQuery.rq", "r") as file:
    agg_query = file.read()
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": agg_query})
assert_response_ok(response, "Failed to run select query.")
print('\n=== Percentage of 5 star ratings ===')
print(response.text)



=== Percentage of 5 star ratings ===
?catalogue	?countOfFiveStarProducts
<https://rdfox.com/example#catalogueXYZ>	2



## Variable scope

Variables in aggregate atoms are local to the atom unless they unless they are group variables (in the `ON` statement).

Therefore, two aggregates can use a variable with the same name so long as it is not a mentioned group variable.

## Where is aggregation relevant?

An incredibly powerful tool, there are unlimited uses of aggregation in production.

COUNT and SUM are so versatile that they can be used in almost any application, with with other functions supporting increasingly targeted and powerful niches.

### Retail

To quantify and analyze user behavior, quantify and rank product sentiment, etc.

### Finance

To sum transactional amounts, isolate extreme values and outliers, validate complex regulation compliance, etc.

### Construction & Manufacturing

To surface and compare high-level compound properties, create a bill of materials, etc.

The list goes on...

## Exercise

Complete the rule `aggregationRules.dlog` in the `rules` folder so that the query below can be used to directly find 'Star Products' - that is, products with a higher than 4 star average rating AND more than 5 total reviews.

### Hits & helpful resources

[Aggregate functions in RDFox](https://docs.oxfordsemantic.tech/querying.html#aggregate-functions)

In [4]:
agg_sparql = """

SELECT ?starProduct ?averageStars ?reviewCount
WHERE {
    ?starProduct a :StarProduct ;
    :hasAverageStars ?averageStars ;
    :hasReviewCount ?reviewCount .
} ORDER BY DESC(?averageStars)

"""

Here is a representative sample of the data in `2_1-AggregationData.ttl`.

In [5]:
sample_data = """
@prefix : <https://rdfox.com/example#> .

:product0001 a :Sofa ;
    :hasReview :review11.

:review11 :hasStars 5 .

"""

### Check your work

Run the query below to verify the results.

In [8]:
# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Get and add data
with open("../data/2_1-AggregationData.ttl", "r") as file:
    aggregation_data = file.read()
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=aggregation_data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get and add rules
with open("../rules/2_1-AggregationRules.dlog", "r") as file:
    aggregation_rules = file.read()
rules_response = requests.post(rdfox_server + "/datastores/default/content", data=aggregation_rules)
assert_response_ok(rules_response, "Failed to add rule.")

# Issue select query
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": agg_sparql})
assert_response_ok(response, "Failed to run select query.")
print('\n=== Star Products ===')
print(response.text)


=== Star Products ===
?starProduct	?averageStars	?reviewCount
<https://rdfox.com/example/product0579>	4.5	8
<https://rdfox.com/example/product0934>	4.5	6
<https://rdfox.com/example/product0395>	4.428571428571428571	7
<https://rdfox.com/example/product0209>	4.375	8
<https://rdfox.com/example/product0317>	4.285714285714285714	7
<https://rdfox.com/example/product0222>	4.166666666666666667	6
<https://rdfox.com/example/product0492>	4.142857142857142857	7
<https://rdfox.com/example/product0931>	4.142857142857142857	7
<https://rdfox.com/example/product0137>	4.142857142857142857	7
<https://rdfox.com/example/product0326>	4.125	8



### Visualise the results

Open this query in the [RDFox Explorer](http://localhost:12110/console/datastores/explore?datastore=default&query=SELECT%20%3FstarProduct%20%3FaverageStars%20%3FreviewCount%0AWHERE%20%7B%0A%20%20%20%20%3FstarProduct%20a%20%3AStarProduct%20%3B%0A%20%20%20%20%3AhasAverageStars%20%3FaverageStars%20%3B%0A%20%20%20%20%3AhasReviewCount%20%3FreviewCount%20.%0A%7D%20ORDER%20BY%20DESC%28%3FaverageStars%29).

## You should see...

=== Star Products ===
|?starProduct|?averageStars|	?reviewCount|
|-----------|-------------|-------------|
|<https://rdfox.com/example/product0579>|	4.5|	8|
|<https://rdfox.com/example/product0934>|	4.5|	6|
|<https://rdfox.com/example/product0395>|	4.428571428571428571|	7|
|<https://rdfox.com/example/product0209>|	4.375|	8|
|<https://rdfox.com/example/product0317>|	4.285714285714285714|	7|
|<https://rdfox.com/example/product0222>|	4.166666666666666667|	6|
|<https://rdfox.com/example/product0931>|	4.142857142857142857|	7|
|<https://rdfox.com/example/product0492>|	4.142857142857142857|	7|
|<https://rdfox.com/example/product0137>|	4.142857142857142857|	7|
|<https://rdfox.com/example/product0326>|	4.125|	8|